https://blog.roboflow.com/train-a-custom-yolov8-pose-estimation-model/

https://www.youtube.com/watch?v=gA5N54IO1ko

In [ ]:
!rm -rf ./runs # удаляем предыдущие результаты

!pip install ultralytics==8.0.196 # рекомендуемая стабильная версия
!pip install roboflow
!pip install torch

Чтобы запустить на сервере:
1. На сервере - `jupyter notebook`
2. На клиенте - `ssh -NfL localhost:ПОРТ:localhost:ПОРТ v.chekhovskii@84.237.51.129 -p 10023`
3. Открываем `localhost:ПОРТ` в браузере
4. Run -> Restart Kernel and Run All Cells... -> Restart

Чтобы остановить проброс портом:
1. На клиенте - `ssh -KL localhost:ПОРТ:localhost:ПОРТ v.chekhovskii@84.237.51.129 -p 10023`

# yolka_train.py

In [ ]:
import ultralytics
from roboflow import Roboflow
from torch.utils.data import DataLoader

from IPython.display import Image, display
import os

Cкачиваем наш датасет с RoboFlow
(https://universe.roboflow.com/mmmm-ooaoaa/yasherki)

In [ ]:
rf = Roboflow(api_key="IJmZ9Rj7mXzHlNQHLSSX") # КЛЮЧ УЖЕ РЕВОУКНУТ, ЕСЛИ ЧТО =)
#project = rf.workspace("object-recognition-ykute").project("horses-mi6xs").version(2)
#project = rf.workspace("capjamesg").project("glue-keypoint-detection").version(1)
#project = rf.workspace("keypoint-test").project("raccoon-pose").version(1)
project = rf.workspace("mmmm-ooaoaa").project("yasherki").version(5)
dataset = project.download("yolov5")

Загружаем yolo-pose
(описание: https://github.com/ultralytics/ultralytics/issues/1915)

In [ ]:
MODEL_NANE = 'yolov8m-pose'
model = ultralytics.YOLO(f'./{MODEL_NANE}.pt')

Тренируем модель

In [ ]:
EPOCHS = 10 # поменять на по болбше
DIR_TRAIN = f'{MODEL_NANE}_trained_{EPOCHS}'
DIR_PRED = f'{MODEL_NANE}_predicted_{EPOCHS}'

In [ ]:
train = model.train(
    data=f'{dataset.location}/data.yaml',
    split='train',
    imgsz=640,
    epochs=EPOCHS,
    batch=16,
    name=DIR_TRAIN
)

~~Сохраняем веса~~
(вместо этого мы используем лучшие(!) веса за все эпохи
из "yolov8n_trained/weights/best.pt",
которые любезно за нас сохраняет YOLOv8)

In [ ]:
#model.save('{MODEL_NANE}_trained')

Демонстрация результатов

In [ ]:
display(Image(f'./runs/pose/{DIR_TRAIN}/confusion_matrix.png', width=600))

In [ ]:
display(Image(f'./runs/pose/{DIR_TRAIN}/results.png', width=800))

# yolka_test.py

Загружаем новую модель с натренированными весами

In [ ]:
#model = ultralytics.YOLO('{MODEL_NANE}_trained')
model = ultralytics.YOLO(f'./runs/pose/{DIR_TRAIN}/weights/best.pt')

Наконец, выполняем предсказания

In [ ]:
pred = model.predict(
    source=f'{dataset.location}/test/images',
    conf=0.40, # 0.25, 0.40, 0.75...
    save=True,
    name=DIR_PRED
)

Демонстрация результатов

In [ ]:
for imgname in os.listdir(f'./runs/pose/{DIR_PRED}'):
    display(Image(f'./runs/pose/{DIR_PRED}/{imgname}', width=200))